In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocessing-video/__results__.html
/kaggle/input/preprocessing-video/video.csv
/kaggle/input/preprocessing-video/__notebook__.ipynb
/kaggle/input/preprocessing-video/course_video.csv
/kaggle/input/preprocessing-video/user_video.csv
/kaggle/input/preprocessing-video/__output__.json
/kaggle/input/preprocessing-video/custom.css
/kaggle/input/translate-school-json-teacher-json/__results__.html
/kaggle/input/translate-school-json-teacher-json/en-teacher.csv
/kaggle/input/translate-school-json-teacher-json/__notebook__.ipynb
/kaggle/input/translate-school-json-teacher-json/en-school.csv
/kaggle/input/translate-school-json-teacher-json/__output__.json
/kaggle/input/translate-school-json-teacher-json/custom.css
/kaggle/input/preprocessing-course-json-2/__results__.html
/kaggle/input/preprocessing-course-json-2/course_field.csv
/kaggle/input/preprocessing-course-json-2/__notebook__.ipynb
/kaggle/input/preprocessing-course-json-2/__output__.json
/kaggle/input/preprocessing-course

In [2]:
course = pd.read_json('/kaggle/input/preprocessing-course-json/course.json',lines=True)
user = pd.read_csv('/kaggle/input/preprocessing-user-json/user-5-100.csv')
user_course = pd.read_csv('/kaggle/input/preprocessing-user-json/user-course.csv')
course_field = pd.read_csv('/kaggle/input/preprocessing-course-json-2/course_field.csv',index_col=None)

In [3]:
school = pd.read_csv('/kaggle/input/preprocessing-school-teacher/school.csv')
teacher = pd.read_csv('/kaggle/input/preprocessing-school-teacher/teacher.csv')
course_school = pd.read_csv('/kaggle/input/mooccubex-relations/course-school.txt',sep='\t',header=None)
course_school_2 = pd.read_csv('/kaggle/input/course-school-inweb/course_school_inweb.csv',header=None)
course_teacher = pd.read_csv('/kaggle/input/mooccubex-relations/course-teacher.txt',sep='\t',header=None)

In [4]:
user_video = pd.read_csv("/kaggle/input/preprocessing-video/user_video.csv")
course_video = pd.read_csv("/kaggle/input/preprocessing-video/course_video.csv")
video = pd.read_csv("/kaggle/input/preprocessing-video/video.csv")

In [5]:
user_exercise = pd.read_csv("/kaggle/input/preprocessing-exercise-1/user_exercise.csv")
course_exercise = pd.read_csv("/kaggle/input/preprocessing-exercise-1/course_exercise.csv")

In [6]:
# comment = 
course_concept = pd.read_csv("/kaggle/input/course-concept/course_concept_prerequisite.csv")
concept = pd.read_csv("/kaggle/input/translated-concept-file/concept_translated.csv")
user_course_comment = pd.read_csv("/kaggle/input/user-course-comment/user_course_comment.csv")

In [7]:
user = user[['id','gender']]
user = user.rename(columns={
    'id':'user_id',
    'gender':'user_gender'
})
user

,user_id,user_gender
0,U_24,1
1,U_53,1
2,U_54,2
3,U_69,1
4,U_90,0
...,...,...
381878,U_34712024,0
381879,U_34712050,1
381880,U_34712098,2
381881,U_34712103,1


In [8]:
course = course[['id','name_trans','prerequisites_trans','about_trans']]
course = course.rename(columns={
    'id':'course_id',
    'name_trans':'course_name',
    'prerequisites_trans':'course_prerequisites',
    'about_trans':'course_about',
})
course.head()

,course_id,course_name,course_prerequisites,course_about
0,C_584313,"introduction to ""zizhi tongjian""",,"through the teacher's guidance, students can g..."
1,C_584329,calculus——extreme theory and univariate function,,this course is a basic mathematics course in s...
2,C_584381,news photography,,"master basic photography skills, understand th..."
3,C_597208,data mining: theory and algorithm,,the most interesting theory + the most useful ...
4,C_597225,university computer,,the university computer courses will be guided...


In [9]:
# xóa những khóa học mẫu, khóa học test (được xác định thủ công)
test_course_id = ['C_1073350','C_1721804','C_1734933','C_1770893','C_1824311',
                  'C_608132','C_645178','C_654397','C_654435','C_681505',
                  'C_789961','C_920114']
course = course[~course['course_id'].isin(test_course_id)]

In [10]:
course_field = course_field[['course_id','course_field']]
course_field = course_field.groupby('course_id').agg(list)
course = pd.merge(course, course_field, on='course_id',how='left')


In [11]:
course['course_field'] = course['course_field'].apply(
    lambda lst: [s.replace('\u200b', '') for s in lst] if isinstance(lst, list) else lst
)
course.head()

,course_id,course_name,course_prerequisites,course_about,course_field
0,C_584313,"introduction to ""zizhi tongjian""",,"through the teacher's guidance, students can g...","[chinese language and literature, history]"
1,C_584329,calculus——extreme theory and univariate function,,this course is a basic mathematics course in s...,"[applied economics, theoretical economics, phy..."
2,C_584381,news photography,,"master basic photography skills, understand th...","[journalism communication, art studies]"
3,C_597208,data mining: theory and algorithm,,the most interesting theory + the most useful ...,[computer science and technology]
4,C_597225,university computer,,the university computer courses will be guided...,NaN


In [12]:
school = school[['id','name_translated','about_translated','motto_translated']]
school = school.rename(columns={
    'id':'school_id',
    'name_translated':'school_name',
    'about_translated':'school_about',
    'motto_translated':'school_motto'
})
school

,school_id,school_name,school_about,school_motto
0,S_1,tsinghua university,referred tsinghua short directly ministry educ...,selfimprovement virtue carry thing
1,S_2,beijing university,peking university referred peking university n...,learn interrogate think carefully distinguish ...
2,S_3,wuhan university,wuhan university wuhan university abbreviated ...,selfreliance hongyi seek truth expand new
3,S_4,suzhou university,soochow university located suzhou famous histo...,nourish righteousness heaven earth law perfect...
4,S_5,sichuan university,sichuan university sichuan university referred...,NaN
...,...,...,...,...
424,S_1402,kaohsiung medical university,kaohsiung medical university selfless donation...,NaN
425,S_1405,heilongjiang agricultural reclamation vocation...,heilongjiang agricultural reclamation vocation...,NaN
426,S_1448,jiangxi university finance economics,jiangxi university finance economics economics...,NaN
427,S_1453,guizhou transportation vocational technical co...,guizhou transportation vocational technical co...,integration knowledge action virtue skill


In [13]:
teacher = teacher[['id','name_translated','about_translated','job_title_translated','org_name_translated']]
teacher = teacher.rename(columns={
    'id':'teacher_id',
    'name_translated':'teacher_name',
    'about_translated':'teacher_about',
    'job_title_translated':'teacher_job_title',
    'org_name_translated':'teacher_org_name'
})
teacher

,teacher_id,teacher_name,teacher_about,teacher_job_title,teacher_org_name
0,T_1,liu yanni,graduated department philosophy peking univers...,lecturer,tsinghua university
1,T_2,chen yi,born chongqing graduated power distribution di...,professor,tsinghua university
2,T_3,cheng gang,cheng gang director course introduction zhuang...,associate professor,tsinghua university
3,T_4,xie weihe,xie weihe doctor professor doctoral supervisor...,professor,tsinghua university
4,T_5,shi jinghuan,shi jinghuan female professor doctoral supervi...,professor,tsinghua university
...,...,...,...,...,...
17013,T_17153,zou wang,graduate changchun automobile industry college...,NaN,changchun automobile industry college
17014,T_17154,wang jianxin,wang jianxin male phd law associate professor ...,associate professor,people public security university china
17015,T_17155,liu xiaodong,liu xiaodong male doctoral master supervisor l...,lecturer,people public security university china
17016,T_17156,zhao,main lecturer course yunnan characteristic fol...,teaching assistant,yunnan academy art


In [14]:
course_school.columns = ['course_id','school_id']
print(course_school.shape)
course_school_2.columns = ['course_id','school_id']
course_school=pd.concat([course_school,course_school_2],axis=0)
course_school=course_school.drop_duplicates()
print(course_school.shape)

(3983, 2)
(4007, 2)


In [15]:
course_teacher.columns = ['course_id','teacher_id']

In [16]:
course.describe()

,course_id,course_name,course_prerequisites,course_about,course_field
count,3769,3769,3769,3769,547
unique,3769,3134,775,3185,156
top,C_584313,linear algebra,,,[computer science and technology]
freq,1,9,2569,87,55


In [17]:
combined_data = pd.merge(user_course, user, on="user_id", how="inner")
combined_data = pd.merge(combined_data,course, on="course_id", how="inner")
combined_data

,user_id,course_id,enroll_time,user_gender,course_name,course_prerequisites,course_about,course_field
0,U_24,C_597314,1560360127,1,university chinese—analysis of northern song d...,,"at present, the talent gap in data science and...",[chinese language and literature]
1,U_24,C_597208,1560784961,1,data mining: theory and algorithm,,the most interesting theory + the most useful ...,[computer science and technology]
2,U_24,C_682129,1570379031,1,software theory basics,,"this course introduces formal language, automa...",NaN
3,U_24,C_674910,1570557064,1,pattern aesthetics and creation,,[national premium course] this course provides...,NaN
4,U_24,C_707038,1570971985,1,organic chemistry,,the world's organic compounds are rich and col...,"[chemical engineering and technology, chemical]"
...,...,...,...,...,...,...,...,...
7473119,U_34712115,C_770738,1602487061,1,data structure (part 2),,breeze plan 2021 [first term recruitment] is i...,NaN
7473120,U_34712115,C_676937,1603225930,1,advanced c++ language programming,,breeze plan 2021 [first term recruitment] is i...,[computer science and technology]
7473121,U_34712115,C_694136,1603226018,1,data mining: theory and algorithm,,the most interesting theory + the most useful ...,NaN
7473122,U_34712115,C_1738993,1603226118,1,game programming,,this course is jointly produced by tsinghua un...,NaN


In [18]:
cb_course_teacher = pd.merge(course_teacher,teacher,on='teacher_id',how='inner')
cb_course_teacher = cb_course_teacher.drop_duplicates(subset=["course_id", "teacher_id"])
cb_course_teacher = cb_course_teacher.groupby(['course_id'])[[
    'teacher_id','teacher_name','teacher_about','teacher_job_title','teacher_org_name']].agg(list).reset_index()
cb_course_teacher

,course_id,teacher_id,teacher_name,teacher_about,teacher_job_title,teacher_org_name
0,C_1001635,"[T_8703, T_8704]","[zhang junwei, yang chao]","[nan, nan]","[nan, nan]",[xian university electronic science technology...
1,C_1007041,[T_9105],[peng dan],[nan],[associate professor],[shenzhen vocational technical college informa...
2,C_1007053,[T_9411],[nie lei],[teacher garden engineering technology],[professor],[shenzhen vocational technical college informa...
3,C_1007072,[T_9411],[nie lei],[teacher garden engineering technology],[professor],[shenzhen vocational technical college informa...
4,C_1007088,[T_9411],[nie lei],[teacher garden engineering technology],[professor],[shenzhen vocational technical college informa...
...,...,...,...,...,...,...
13861,C_999396,"[T_7592, T_7593, T_7594]","[zhang yaling, lu xiaofeng, wang zhanmin]",[course leader main lecturer professor zhang y...,"[professor, associate professor, associate pro...","[xian university technology, xian university t..."
13862,C_999397,"[T_4524, T_4525, T_4526]","[li shugang, cheng lianhua, liu jikun]",[li shugang phd engineering professor doctoral...,"[professor, professor, associate professor]","[xian university science technology, xian univ..."
13863,C_999399,"[T_871, T_204, T_7596, T_7597, T_7598, T_7599,...","[wang hong, wang yuan, first teacher name, she...",[wang hong master professor master supervisor ...,"[professor, librarian, x, lecturer, lecturer, ...","[kunming medical university, tsinghua universi..."
13864,C_999403,"[T_871, T_204, T_7596, T_7597, T_7598, T_7599,...","[wang hong, wang yuan, first teacher name, she...",[wang hong master professor master supervisor ...,"[professor, librarian, x, lecturer, lecturer, ...","[kunming medical university, tsinghua universi..."


In [19]:
cb_course_school = pd.merge(course_school,school,on='school_id',how='inner')
cols_name = school.columns
cb_course_school = cb_course_school.drop_duplicates()
cb_course_school = cb_course_school.groupby(['course_id'])[cols_name].agg(list).reset_index()
cb_course_school

,course_id,school_id,school_name,school_about,school_motto
0,C_1017355,[S_33],[guangzhou university traditional chinese medi...,[guangzhou university chinese medicine first b...,[nan]
1,C_1017419,[S_52],[central south university nationality],[southcentral minzu university referred scmzu ...,[nan]
2,C_1025064,[S_13],[central south university],[central south university located changsha cit...,[integration knowledge action practice world]
3,C_1025076,[S_72],[northeastern university],[northeastern university northeastern universi...,[nan]
4,C_1025079,[S_97],[beijing sport university],[beijing sport university referred beijing spo...,[nan]
...,...,...,...,...,...
3811,C_955163,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing]
3812,C_956128,[S_144],[shandong university],[shandong university directly ministry educati...,[nan]
3813,C_956129,[S_492],[guangzhou vocational college science technology],[guangzhou science technology trade vocational...,[nan]
3814,C_956130,[S_11],[south china university technology],[south china university technology referred hu...,[learn think carefully clear clear]


In [20]:
video=video.rename(columns={'ccid':'video_ccid'})

In [21]:
video.isnull().sum()

video_id          0
video_ccid        0
video_duration    0
dtype: int64

In [22]:
video.isnull().mean()

video_id          0.0
video_ccid        0.0
video_duration    0.0
dtype: float64

In [23]:
cb_course_video = pd.merge(course_video,video,on='video_id',how="left")
cb_course_video.isnull().mean()

course_id         0.0
video_id          0.0
video_ccid        0.0
video_duration    0.0
dtype: float64

In [24]:
cols_name = video.columns
cb_course_video = cb_course_video.drop_duplicates()
cb_course_video = cb_course_video.dropna()
cb_course_video_list = cb_course_video.groupby(['course_id'])[cols_name].agg(list).reset_index()
cb_course_video_list

,course_id,video_id,video_ccid,video_duration
0,C_1025064,"[V_1622637, V_1622640, V_1622622, V_1622571, V...","[0B63D5E3D4D034E89C33DC5901307461, 0B6DEBC0F80...","[466.575, 402.325, 476.65, 1025.9, 755.075, 42..."
1,C_1025079,"[V_1623000, V_1622990, V_1622999, V_1623003, V...","[0805D199A89E1D799C33DC5901307461, 1026CDC4B72...","[671.97, 925.17, 748.11, 644.25, 543.93, 546.3..."
2,C_1073350,"[V_1666680, V_4903099]","[85AED0EB3E502F579C33DC5901307461, C89B9BCF178...","[537.42, 353.135]"
3,C_1123814,"[V_1685285, V_1685292, V_1685305, V_1685268, V...","[037ADF4715C21A839C33DC5901307461, 09ABCD91037...","[479.741, 629.348, 563.919, 682.89, 634.135, 8..."
4,C_1169394,"[V_1724743, V_1724514, V_1724866, V_1724794, V...","[0086579E5E48FC179C33DC5901307461, 06952EAE1BB...","[649.72, 304.4, 193.64, 493.92, 372.2, 468.04,..."
...,...,...,...,...
1517,C_949540,"[V_2316648, V_2316654, V_3183769, V_3462935, V...","[00CC2D694D4C9AD99C33DC5901307461, 00F9DB5674F...","[702.625, 754.8, 727.15, 673.1, 379.586, 977.3..."
1518,C_949541,"[V_1505096, V_1505098, V_1505056, V_1505102, V...","[0A19E8C9397E5A879C33DC5901307461, 10022BA057E...","[465.175, 398.82, 560.31, 484.5, 632.925, 827...."
1519,C_949542,"[V_1505228, V_1505190, V_1505250, V_1505241, V...","[009BB91833DD44A49C33DC5901307461, 012B7F50301...","[784.77, 713.7, 578.07, 689.01, 624.45, 658.17..."
1520,C_955163,"[V_1542395, V_1542782, V_1542410, V_1542374, V...","[0314DA139E06D2CA9C33DC5901307461, 0A4FA12E2B7...","[515.893, 134.584, 2084.548, 1546.264, 38.356,..."


In [25]:
user_video

,user_id,video_id,total_watch_time,actual_watch_time
0,U_15767679,V_6340530,388.174,388.174
1,U_16097788,V_1395635,59.700,59.700
2,U_33771074,V_7592810,645.000,645.000
3,U_32440729,V_7472597,340.000,340.000
4,U_19842780,V_1358727,25.000,25.000
...,...,...,...,...
1980996,U_30211589,V_5558657,245.000,245.000
1980997,U_30310046,V_5149580,218.400,216.800
1980998,U_9582167,V_7857876,189.900,189.900
1980999,U_15285678,V_6021330,384.300,384.300


In [26]:
cb_user_course_video = pd.merge(user_video,cb_course_video,on=['video_id'],how='right')
cb_user_course_video

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,user_id,video_id,total_watch_time,actual_watch_time,course_id,video_ccid,video_duration
0,NaN,V_93474,NaN,NaN,C_677105,0001603F826A3DDA9C33DC5901307461,509.095
1,NaN,V_472954,NaN,NaN,C_696877,0003DB14A14A53279C33DC5901307461,367.336
2,NaN,V_448757,NaN,NaN,C_696659,0004A5C6F07E369D9C33DC5901307461,492.315
3,NaN,V_5080653,NaN,NaN,C_1784725,0004A5C6F07E369D9C33DC5901307461,492.315
4,NaN,V_555489,NaN,NaN,C_697721,00059EBD1371A6959C33DC5901307461,630.757
...,...,...,...,...,...,...,...
121752,NaN,V_5987059,NaN,NaN,C_1886688,FFF99B018E97FA6E9C33DC5901307461,784.400
121753,NaN,V_140144,NaN,NaN,C_680915,FFFB9B63E30B25919C33DC5901307461,542.000
121754,NaN,V_220033,NaN,NaN,C_681746,FFFE61FFCECC05729C33DC5901307461,554.400
121755,NaN,V_275285,NaN,NaN,C_682190,FFFE61FFCECC05729C33DC5901307461,554.400


In [27]:
cols_name = ['video_id','total_watch_time','actual_watch_time','video_ccid','video_duration']
cb_user_course_video = cb_user_course_video.rename(columns={'ccid':'video_ccid'})
cb_user_course_video = cb_user_course_video.groupby(['user_id','course_id'])[cols_name].agg(list).reset_index()
cb_user_course_video

,user_id,course_id,video_id,total_watch_time,actual_watch_time,video_ccid,video_duration
0,U_1000454,C_2199449,[V_7396061],[450.081],[450.081],[05F564395C35232E9C33DC5901307461],[462.142]
1,U_1002784,C_1756056,"[V_4411676, V_4411678, V_4411680]","[350.20000000000005, 285.40000000000003, 550.0]","[350.20000000000005, 285.40000000000003, 550.0]","[439EAFC2EA7521699C33DC5901307461, 7BACA405FA1...","[379.35, 284.32, 896.41]"
2,U_100281,C_1764866,[V_4677748],[416.1],[452.1],[15E18AD98720C9B89C33DC5901307461],[444.819]
3,U_100281,C_747033,"[V_731722, V_731719, V_731716]","[150.0, 486.0, 62.0]","[150.0, 471.0, 62.0]","[259709F4A24A3B4F9C33DC5901307461, 4CE60DF8B54...","[1040.397, 472.05, 68.57]"
4,U_10033817,C_676642,"[V_1642603, V_1642632, V_1642631]","[10.0, 10.0, 5.0]","[10.0, 10.0, 5.0]","[5846BB9A01701AE89C33DC5901307461, 9BF9C86E729...","[343.2, 370.0, 823.64]"
...,...,...,...,...,...,...,...
11926,U_9968019,C_769273,"[V_768979, V_768978, V_768976, V_768980]","[120.0, 575.0, 101.0, 149.0]","[120.0, 559.0, 86.0, 149.0]","[08D27E716C0B70F89C33DC5901307461, 15E1F223F1C...","[650.675, 643.7, 356.65, 807.6]"
11927,U_996997,C_1774987,[V_4886058],[38.30000000000001],[38.30000000000001],[452016DB4F8D62199C33DC5901307461],[254.001]
11928,U_996997,C_735164,"[V_6293309, V_6293303, V_6293306]","[162.79999999999995, 482.6, 380.0]","[162.79999999999995, 482.6, 380.0]","[9A0D280494C185A49C33DC5901307461, B5FD17CF9EA...","[693.575, 608.955, 799.13]"
11929,U_997041,C_735164,"[V_6293456, V_6293303, V_6293306, V_6293280]","[185.221684, 429.369029, 354.74286299999994, 4...","[185.221684, 429.369029, 354.74286299999994, 4...","[6F5009A0ECE24F0D9C33DC5901307461, B5FD17CF9EA...","[255.616, 608.955, 799.13, 857.747]"


In [28]:
cb_course = pd.merge(course,cb_course_school,on='course_id',how='left')
cb_course = pd.merge(cb_course,cb_course_teacher,on='course_id',how='left')
cb_course = pd.merge(cb_course,cb_course_video_list,on='course_id',how='left')
# cb_course = pd.merge(cb_course,cb_course_exercise,on='course_id',how='left')
cb_course

,course_id,course_name,course_prerequisites,course_about,course_field,school_id,school_name,school_about,school_motto,teacher_id,teacher_name,teacher_about,teacher_job_title,teacher_org_name,video_id,video_ccid,video_duration
0,C_584313,"introduction to ""zizhi tongjian""",,"through the teacher's guidance, students can g...","[chinese language and literature, history]",[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],[T_20],[zhang guogang],[zhang guogang male born anqing anhui doctoral...,[professor],[tsinghua university],"[V_867, V_931, V_952, V_901, V_857, V_915, V_8...","[0192C9E042651D369C33DC5901307461, 02B1CA99EAE...","[524.371, 397.52, 677.391, 997.872, 821.917, 6..."
1,C_584329,calculus——extreme theory and univariate function,,this course is a basic mathematics course in s...,"[applied economics, theoretical economics, phy...",[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],"[T_21, T_22]","[hu zhiming, zhang jimin]",[graduated department mathematics peking unive...,"[associate professor, associate professor]","[tsinghua university, tsinghua university]","[V_1525, V_1473, V_1470, V_1497, V_1449, V_146...","[01025FA43C787FF09C33DC5901307461, 01ABB469C7E...","[585.519, 446.6, 1216.519, 789.1, 1314.28, 543..."
2,C_584381,news photography,,"master basic photography skills, understand th...","[journalism communication, art studies]",[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],[T_23],[liang junjian],[associate professor school journalism communi...,[associate professor],[tsinghua university],"[V_1822, V_1806, V_1896, V_1869, V_1901, V_186...","[01D0B67D2154D3459C33DC5901307461, 01F0AEBFB50...","[444.56, 204.613, 505.287, 606.654, 700.328, 2..."
3,C_597208,data mining: theory and algorithm,,the most interesting theory + the most useful ...,[computer science and technology],[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],[T_28],[yuan bo],[doctor computer science university queensland...,[associate researcher],[tsinghua university],"[V_3025, V_2985, V_2964, V_3074, V_3082, V_306...","[03D205A6AEB0B16F9C33DC5901307461, 03F5AF209FA...","[1003.9, 773.66, 886.35, 505.57, 889.61, 930.6..."
4,C_597225,university computer,,the university computer courses will be guided...,NaN,[S_6],[beijing institute technology],[beijing institute technology first science en...,[nan],"[T_41, T_42]","[li fengxia, chen yufeng]",[li fengxia female professor school computer s...,"[professor, associate professor]","[beijing institute technology, beijing institu...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3764,C_2338076,(breathing plan) software engineering,,breeze plan 2021 [first term recruitment] is i...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],"[T_172, T_835]","[liu qiang, liu lin]",[liu qiang associate professor graduate superv...,"[associate professor, associate professor]","[tsinghua university, tsinghua university]","[V_8729262, V_8729306, V_8729264, V_8729305, V...","[0213690C975015059C33DC5901307461, 0482FE3877C...","[1108.66, 391.7, 777.06, 729.02, 723.62, 382.2..."
3765,C_2341259,(breathing project) object-oriented programmin...,,breeze plan 2021 [first term recruitment] is i...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],"[T_38, T_3160]","[xu mingxing, huang zhenchun]",[xu mingxing phd associate professor person ch...,"[associate professor, associate researcher]","[tsinghua university, tsinghua university]","[V_8782385, V_8782393, V_8782394, V_8782392, V...","[05E8B15BD7931AEC9C33DC5901307461, 13E87E6923F...","[439.653, 470.705, 669.787, 806.114, 350.802, ..."
3766,C_2337996,(breath pla

In [29]:
combine_data = pd.merge(user_course,cb_course,on='course_id',how='inner')
combine_data

,user_id,course_id,enroll_time,course_name,course_prerequisites,course_about,course_field,school_id,school_name,school_about,school_motto,teacher_id,teacher_name,teacher_about,teacher_job_title,teacher_org_name,video_id,video_ccid,video_duration
0,U_24,C_597314,1560360127,university chinese—analysis of northern song d...,,"at present, the talent gap in data science and...",[chinese language and literature],"[S_1, S_14]","[tsinghua university, southeast university]",[referred tsinghua short directly ministry edu...,"[selfimprovement virtue carry thing, stop high...",[T_113],[wang bugao],[wang bugao yangzhong jiangsu studied nanjing ...,[distinguished professor],[tsinghua university],"[V_10741, V_10729, V_10669, V_10636, V_10583, ...","[059F9E3D0C61EC3A9C33DC5901307461, 05BA5BEC636...","[678.88, 1861.28, 280.63, 1488.369, 1192.15, 6..."
1,U_24,C_597208,1560784961,data mining: theory and algorithm,,the most interesting theory + the most useful ...,[computer science and technology],[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],[T_28],[yuan bo],[doctor computer science university queensland...,[associate researcher],[tsinghua university],"[V_3025, V_2985, V_2964, V_3074, V_3082, V_306...","[03D205A6AEB0B16F9C33DC5901307461, 03F5AF209FA...","[1003.9, 773.66, 886.35, 505.57, 889.61, 930.6..."
2,U_24,C_682129,1570379031,software theory basics,,"this course introduces formal language, automa...",NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],[T_2648],[luo guiming],[luo guiming received master degree university...,[professor],[tsinghua university],"[V_265396, V_265411, V_265404, V_265442, V_265...","[00784179599D8C659C33DC5901307461, 036F26D5314...","[1236.79, 1212.67, 1584.55, 1574.35, 376.79, 5..."
3,U_24,C_674910,1570557064,pattern aesthetics and creation,,[national premium course] this course provides...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],[T_307],[nie yuehua],[nie yuehua member chinese artist association ...,[associate professor],[tsinghua university],NaN,NaN,NaN
4,U_24,C_707038,1570971985,organic chemistry,,the world's organic compounds are rich and col...,"[chemical engineering and technology, chemical]",[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],"[T_999, T_1000, T_1001, T_1002, T_1003, T_1004...","[li yanmei, li zhaolong, hua ruimao, fu hua, z...",[li yanmei professor doctoral supervisor recei...,"[professor, researcher, professor, professor, ...","[tsinghua university, tsinghua university, tsi...","[V_566855, V_567087, V_566826, V_567004, V_566...","[001EF0F9EA519E1B9C33DC5901307461, 003EB4E2CD2...","[194.2, 295.6, 423.465, 270.269, 529.6, 183.69..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7473119,U_34712115,C_770738,1602487061,data structure (part 2),,breeze plan 2021 [first term recruitment] is i...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],[T_805],[deng junhui],[deng junhui professor department computer sci...,[professor],[tsinghua university],"[V_784218, V_784314, V_784095, V_784020, V_784...","[00CE59849A6B87B29C33DC5901307461, 00DDC85FD9E...","[173.09, 312.0, 333.86, 446.9, 93.46, 142.56, ..."
7473120,U_34712115,C_676937,1603225930,advanced c++ language programming,,breeze plan 2021 [first term recruitment] is i...,[computer science and technology],[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],"[T_540, T_541, T_38]","[zheng li, li chao, xu mingxing]",[zheng li professor department computer scienc...,"[professor, associate researcher, associate pr...","[tsinghua university, tsinghua university, tsi...","[V_69237, V_69353, V_69285, V_69228, V_69217, .

In [30]:
combine_data = pd.merge(combine_data,user,on='user_id',how='inner')
combine_data

,user_id,course_id,enroll_time,course_name,course_prerequisites,course_about,course_field,school_id,school_name,school_about,school_motto,teacher_id,teacher_name,teacher_about,teacher_job_title,teacher_org_name,video_id,video_ccid,video_duration,user_gender
0,U_24,C_597314,1560360127,university chinese—analysis of northern song d...,,"at present, the talent gap in data science and...",[chinese language and literature],"[S_1, S_14]","[tsinghua university, southeast university]",[referred tsinghua short directly ministry edu...,"[selfimprovement virtue carry thing, stop high...",[T_113],[wang bugao],[wang bugao yangzhong jiangsu studied nanjing ...,[distinguished professor],[tsinghua university],"[V_10741, V_10729, V_10669, V_10636, V_10583, ...","[059F9E3D0C61EC3A9C33DC5901307461, 05BA5BEC636...","[678.88, 1861.28, 280.63, 1488.369, 1192.15, 6...",1
1,U_24,C_597208,1560784961,data mining: theory and algorithm,,the most interesting theory + the most useful ...,[computer science and technology],[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],[T_28],[yuan bo],[doctor computer science university queensland...,[associate researcher],[tsinghua university],"[V_3025, V_2985, V_2964, V_3074, V_3082, V_306...","[03D205A6AEB0B16F9C33DC5901307461, 03F5AF209FA...","[1003.9, 773.66, 886.35, 505.57, 889.61, 930.6...",1
2,U_24,C_682129,1570379031,software theory basics,,"this course introduces formal language, automa...",NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],[T_2648],[luo guiming],[luo guiming received master degree university...,[professor],[tsinghua university],"[V_265396, V_265411, V_265404, V_265442, V_265...","[00784179599D8C659C33DC5901307461, 036F26D5314...","[1236.79, 1212.67, 1584.55, 1574.35, 376.79, 5...",1
3,U_24,C_674910,1570557064,pattern aesthetics and creation,,[national premium course] this course provides...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],[T_307],[nie yuehua],[nie yuehua member chinese artist association ...,[associate professor],[tsinghua university],NaN,NaN,NaN,1
4,U_24,C_707038,1570971985,organic chemistry,,the world's organic compounds are rich and col...,"[chemical engineering and technology, chemical]",[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],"[T_999, T_1000, T_1001, T_1002, T_1003, T_1004...","[li yanmei, li zhaolong, hua ruimao, fu hua, z...",[li yanmei professor doctoral supervisor recei...,"[professor, researcher, professor, professor, ...","[tsinghua university, tsinghua university, tsi...","[V_566855, V_567087, V_566826, V_567004, V_566...","[001EF0F9EA519E1B9C33DC5901307461, 003EB4E2CD2...","[194.2, 295.6, 423.465, 270.269, 529.6, 183.69...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7473119,U_34712115,C_770738,1602487061,data structure (part 2),,breeze plan 2021 [first term recruitment] is i...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],[T_805],[deng junhui],[deng junhui professor department computer sci...,[professor],[tsinghua university],"[V_784218, V_784314, V_784095, V_784020, V_784...","[00CE59849A6B87B29C33DC5901307461, 00DDC85FD9E...","[173.09, 312.0, 333.86, 446.9, 93.46, 142.56, ...",1
7473120,U_34712115,C_676937,1603225930,advanced c++ language programming,,breeze plan 2021 [first term recruitment] is i...,[computer science and technology],[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,[selfimprovement virtue carry thing],"[T_540, T_541, T_38]","[zheng li, li chao, xu mingxing]",[zheng li professor department computer scienc...,"[professor, associate researcher, associate pr...","[tsinghua university, tsinghua university, tsi...","[V_69237, V_69353, 

In [31]:
user_course_video = pd.merge(user_video,course_video[['course_id','video_id']],on='video_id',how='inner')
user_course_video = user_course_video.groupby(['user_id','course_id']).agg(list).reset_index()
user_course_video = user_course_video.rename(columns = {'video_id':'watch_id'})
user_course_video

,user_id,course_id,watch_id,total_watch_time,actual_watch_time
0,U_1000454,C_2199449,[V_7396061],[450.081],[450.081]
1,U_1002784,C_1756056,"[V_4411680, V_4411678, V_4411676]","[550.0, 285.40000000000003, 350.20000000000005]","[550.0, 285.40000000000003, 350.20000000000005]"
2,U_100281,C_1764866,[V_4677748],[416.1],[452.1]
3,U_100281,C_747033,"[V_731719, V_731722, V_731716]","[486.0, 150.0, 62.0]","[471.0, 150.0, 62.0]"
4,U_10033817,C_676642,"[V_1642632, V_1642603, V_1642631]","[10.0, 10.0, 5.0]","[10.0, 10.0, 5.0]"
...,...,...,...,...,...
11926,U_9968019,C_769273,"[V_768979, V_768976, V_768980, V_768978]","[120.0, 101.0, 149.0, 575.0]","[120.0, 86.0, 149.0, 559.0]"
11927,U_996997,C_1774987,[V_4886058],[38.30000000000001],[38.30000000000001]
11928,U_996997,C_735164,"[V_6293306, V_6293303, V_6293309]","[380.0, 482.6, 162.79999999999995]","[380.0, 482.6, 162.79999999999995]"
11929,U_997041,C_735164,"[V_6293306, V_6293456, V_6293280, V_6293303]","[354.74286299999994, 185.221684, 499.509613000...","[354.74286299999994, 185.221684, 499.509613000..."


In [32]:
combine_data = pd.merge(combine_data,user_course_video,on=["user_id","course_id"],how='left')
combine_data

,user_id,course_id,enroll_time,course_name,course_prerequisites,course_about,course_field,school_id,school_name,school_about,...,teacher_about,teacher_job_title,teacher_org_name,video_id,video_ccid,video_duration,user_gender,watch_id,total_watch_time,actual_watch_time
0,U_24,C_597314,1560360127,university chinese—analysis of northern song d...,,"at present, the talent gap in data science and...",[chinese language and literature],"[S_1, S_14]","[tsinghua university, southeast university]",[referred tsinghua short directly ministry edu...,...,[wang bugao yangzhong jiangsu studied nanjing ...,[distinguished professor],[tsinghua university],"[V_10741, V_10729, V_10669, V_10636, V_10583, ...","[059F9E3D0C61EC3A9C33DC5901307461, 05BA5BEC636...","[678.88, 1861.28, 280.63, 1488.369, 1192.15, 6...",1,NaN,NaN,NaN
1,U_24,C_597208,1560784961,data mining: theory and algorithm,,the most interesting theory + the most useful ...,[computer science and technology],[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,[doctor computer science university queensland...,[associate researcher],[tsinghua university],"[V_3025, V_2985, V_2964, V_3074, V_3082, V_306...","[03D205A6AEB0B16F9C33DC5901307461, 03F5AF209FA...","[1003.9, 773.66, 886.35, 505.57, 889.61, 930.6...",1,NaN,NaN,NaN
2,U_24,C_682129,1570379031,software theory basics,,"this course introduces formal language, automa...",NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,[luo guiming received master degree university...,[professor],[tsinghua university],"[V_265396, V_265411, V_265404, V_265442, V_265...","[00784179599D8C659C33DC5901307461, 036F26D5314...","[1236.79, 1212.67, 1584.55, 1574.35, 376.79, 5...",1,NaN,NaN,NaN
3,U_24,C_674910,1570557064,pattern aesthetics and creation,,[national premium course] this course provides...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,[nie yuehua member chinese artist association ...,[associate professor],[tsinghua university],NaN,NaN,NaN,1,NaN,NaN,NaN
4,U_24,C_707038,1570971985,organic chemistry,,the world's organic compounds are rich and col...,"[chemical engineering and technology, chemical]",[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,[li yanmei professor doctoral supervisor recei...,"[professor, researcher, professor, professor, ...","[tsinghua university, tsinghua university, tsi...","[V_566855, V_567087, V_566826, V_567004, V_566...","[001EF0F9EA519E1B9C33DC5901307461, 003EB4E2CD2...","[194.2, 295.6, 423.465, 270.269, 529.6, 183.69...",1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7473119,U_34712115,C_770738,1602487061,data structure (part 2),,breeze plan 2021 [first term recruitment] is i...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,[deng junhui professor department computer sci...,[professor],[tsinghua university],"[V_784218, V_784314, V_784095, V_784020, V_784...","[00CE59849A6B87B29C33DC5901307461, 00DDC85FD9E...","[173.09, 312.0, 333.86, 446.9, 93.46, 142.56, ...",1,NaN,NaN,NaN
7473120,U_34712115,C_676937,1603225930,advanced c++ language programming,,breeze plan 2021 [first term recruitment] is i...,[computer science and technology],[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,[zheng li professor department computer scienc...,"[professor, associate researcher, associate pr...","[tsinghua university, tsinghua university, tsi...","[V_69237, V_69353, V_69285, V_69228, V_69217, ...","[0086579E5E48FC179C33DC5901307461, 0814FA18F28...","[649.72, 193.64, 493.92, 372.2, 468.04, 230.24...",1,NaN,NaN,NaN
7473121,U_34712115,C_694136,1603226018,data mining: theory and algorithm,,the most interesting theory + the most useful ...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,[doctor computer science university queensland...,[associate rese

In [33]:
course_exercise

,course_id,exercise_id,total_pm_id
0,C_883372,Ex_2229164,1
1,C_1764857,Ex_4676558,30
2,C_696945,Ex_870215,2
3,C_1875503,Ex_5934951,6
4,C_756654,Ex_752940,6
...,...,...,...
66699,C_734024,Ex_760617,18
66700,C_682297,Ex_294244,3
66701,C_682158,Ex_269331,1
66702,C_876705,Ex_1165821,10


In [34]:
user_exercise = user_exercise.rename(columns = {'num_pm_id':'doing_num_correct'})
course_exercise = course_exercise.rename(columns = {'total_pm_id':'exercise_num_problem'})

In [35]:
user_course_exercise = pd.merge(user_exercise,course_exercise[['course_id','exercise_id']],on='exercise_id',how='inner')
user_course_exercise = user_course_exercise.rename(columns = {'exercise_id':'doing_exercise_id'})
user_course_exercise

,user_id,doing_exercise_id,doing_num_correct,course_id
0,U_18305915,Ex_1510708,10,C_948488
1,U_12575652,Ex_1365029,10,C_735214
2,U_4661678,Ex_314248,3,C_682392
3,U_11700672,Ex_5667255,3,C_1822804
4,U_22713340,Ex_8157840,15,C_2287011
...,...,...,...,...
2549737,U_30913159,Ex_5667250,5,C_1822804
2549738,U_12648695,Ex_1384807,3,C_845950
2549739,U_30944706,Ex_5667254,3,C_1822804
2549740,U_26006951,Ex_1398682,5,C_893222


In [36]:
course_exercise

,course_id,exercise_id,exercise_num_problem
0,C_883372,Ex_2229164,1
1,C_1764857,Ex_4676558,30
2,C_696945,Ex_870215,2
3,C_1875503,Ex_5934951,6
4,C_756654,Ex_752940,6
...,...,...,...
66699,C_734024,Ex_760617,18
66700,C_682297,Ex_294244,3
66701,C_682158,Ex_269331,1
66702,C_876705,Ex_1165821,10


In [37]:
cols_name = course_exercise.columns
cb_course_exercise = course_exercise.groupby(['course_id']).agg(list).reset_index()
cb_course_exercise

,course_id,exercise_id,exercise_num_problem
0,C_1017355,"[Ex_1617861, Ex_1736572, Ex_1617873, Ex_161788...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
1,C_1017419,"[Ex_1618263, Ex_1618257, Ex_1618248, Ex_161826...","[15, 15, 20, 15, 15, 15, 15, 16, 15]"
2,C_1025064,"[Ex_1622651, Ex_1622617, Ex_1622586, Ex_162263...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1..."
3,C_1025076,"[Ex_1622870, Ex_1622876, Ex_1622875, Ex_1622889]","[1, 1, 15, 1]"
4,C_1025079,"[Ex_1623011, Ex_1622993, Ex_1623006, Ex_162302...","[5, 5, 5, 5, 5]"
...,...,...,...
3196,C_955163,"[Ex_1545426, Ex_1576376, Ex_1545423, Ex_154542...","[5, 1, 10, 10, 1, 30, 4]"
3197,C_956128,"[Ex_1555008, Ex_1555014, Ex_1555038, Ex_155504...","[15, 15, 15, 15, 15, 15, 15]"
3198,C_956129,"[Ex_1555128, Ex_1555206, Ex_1555192, Ex_155513...","[2, 3, 3, 8, 7, 6, 10, 9, 9, 6, 4, 1, 6, 7, 8,..."
3199,C_956130,"[Ex_6304361, Ex_6304318, Ex_7777127, Ex_630438...","[1, 5, 27, 8, 6, 19, 28, 6, 14, 16, 21, 6, 15,..."


In [38]:
cols_name = user_course_exercise.columns
user_course_exercise = user_course_exercise.groupby(['user_id','course_id']).agg(list).reset_index()
user_course_exercise

,user_id,course_id,doing_exercise_id,doing_num_correct
0,U_10000,C_2033958,[Ex_7007033],[1]
1,U_1000454,C_2199449,[Ex_7396062],[3]
2,U_100090,C_949542,[Ex_1505248],[1]
3,U_1000979,C_947149,"[Ex_1431537, Ex_1431541, Ex_1431547, Ex_143154...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5]"
4,U_1000982,C_947149,"[Ex_1431543, Ex_1431551, Ex_1431545, Ex_1431547]","[3, 1, 1, 2]"
...,...,...,...,...
158393,U_99753,C_1428968,"[Ex_6381948, Ex_6381972, Ex_6381959, Ex_642221...","[7, 7, 10, 9, 8]"
158394,U_997542,C_2066096,"[Ex_7413710, Ex_7604461, Ex_7280076, Ex_740428...","[17, 21, 18, 25, 25]"
158395,U_99772,C_1903985,"[Ex_6179347, Ex_6179343, Ex_6179337, Ex_617934...","[6, 16, 8, 9, 34, 8, 8]"
158396,U_9988,C_1765600,"[Ex_4692038, Ex_4692022, Ex_4692036]","[1, 1, 1]"


In [39]:
combine_data = pd.merge(combine_data,cb_course_exercise,on='course_id',how='left')
combine_data = pd.merge(combine_data,user_course_exercise,on=['user_id','course_id'],how='left')

In [40]:
course_concept = course_concept[['concept_id','ccid','course_id','c1']]
course_concept = course_concept.rename(columns={'ccid':'concept_ccid','c1':'concept_prerequisite'})
course_concept.head()

,concept_id,concept_ccid,course_id,concept_prerequisite
0,K_001定时器得电_控制科学与工程,6EE6CD9AA5B407709C33DC5901307461,C_681460,NaN
1,K_002平面衍射强度_作物学,1260AEADBA186B9D9C33DC5901307461,C_697288,NaN
2,K_0 part_计算机科学与技术,03D205A6AEB0B16F9C33DC5901307461,C_597208,NaN
3,K_0 part_计算机科学与技术,03D205A6AEB0B16F9C33DC5901307461,C_694136,NaN
4,K_0 part_计算机科学与技术,03D205A6AEB0B16F9C33DC5901307461,C_830568,NaN


In [41]:
concept.head()

,concept_id,concept_name,concept_field,concept_name_translated,concept_field_translated
0,K_神经部_组织学与胚胎学,神经部,组织学与胚胎学,nervous system,histology and embryology
1,K_促甲状腺激素细胞_组织学与胚胎学,促甲状腺激素细胞,组织学与胚胎学,tsh cells,histology and embryology
2,K_嗜色细胞_组织学与胚胎学,嗜色细胞,组织学与胚胎学,chromophilic cells,histology and embryology
3,K_生长激素细胞_组织学与胚胎学,生长激素细胞,组织学与胚胎学,growth hormone cells,histology and embryology
4,K_褐铁矿_材料科学与工程,褐铁矿,材料科学与工程,limonite,materials science and engineering


In [42]:
concept = concept[['concept_id','concept_name_translated','concept_field_translated']].rename(columns={
    'concept_name_translated':'concept_name',
    'concept_field_translated':'concept_field'
})
concept

,concept_id,concept_name,concept_field
0,K_神经部_组织学与胚胎学,nervous system,histology and embryology
1,K_促甲状腺激素细胞_组织学与胚胎学,tsh cells,histology and embryology
2,K_嗜色细胞_组织学与胚胎学,chromophilic cells,histology and embryology
3,K_生长激素细胞_组织学与胚胎学,growth hormone cells,histology and embryology
4,K_褐铁矿_材料科学与工程,limonite,materials science and engineering
...,...,...,...
637569,K_纤维结肠镜_食品科学与工程,fiber colonoscopy,food science and engineering
637570,K_齐鲁工业大学_食品科学与工程,qilu university of technology,food science and engineering
637571,K_杞 _食品科学与工程,qi,food science and engineering
637572,K_转基因玉米_食品科学与工程,genetically modified corn,food science and engineering


In [43]:
concept[concept['concept_field'].isnull()|concept['concept_name'].isnull()].reset_index(drop=True)

,concept_id,concept_name,concept_field
0,K_φ,NaN,NaN
1,扫描_化学,NaN,NaN
2,扫描,chemical,NaN
3,K_nan _计算机科学与技术,NaN,computer science and technology


In [44]:
to_remove = ['K_φ', '扫描']
concept = concept[~concept['concept_id'].isin(to_remove)]
concept.loc[concept['concept_id'] == 'K_nan _计算机科学与技术', 'concept_name'] = 'nan value'
# Thêm 'K_' vào đầu concept_id
concept.loc[concept['concept_id'] == '扫描_化学', 'concept_id'] = 'K_扫描_化学'
# Thay NaN ở concept_name bằng 'scan'
concept.loc[concept['concept_id'] == 'K_扫描_化学', 'concept_name'] = 'scan'
# Thay NaN ở concept_field bằng 'chemistry'
concept.loc[concept['concept_id'] == 'K_扫描_化学', 'concept_field'] = 'chemistry'

<ipython-input-44-091fa37ee009>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concept.loc[concept['concept_id'] == 'K_nan _计算机科学与技术', 'concept_name'] = 'nan value'
<ipython-input-44-091fa37ee009>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concept.loc[concept['concept_id'] == '扫描_化学', 'concept_id'] = 'K_扫描_化学'
<ipython-input-44-091fa37ee009>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concept.loc[concept['concept_id'] == '

In [45]:
user_course_comment

,user_id,course_id,num_comments,num_replies,total_interactions,total_course_comments
0,U_24,C_597214,0,0,0,0
1,U_24,C_605512,0,0,0,1
2,U_24,C_597211,0,0,0,0
3,U_24,C_597314,0,0,0,0
4,U_24,C_597208,0,0,0,0
...,...,...,...,...,...,...
7591770,U_34712115,C_770738,0,0,0,339
7591771,U_34712115,C_676937,0,0,0,370
7591772,U_34712115,C_694136,0,0,0,1108
7591773,U_34712115,C_1738993,0,0,0,77


In [46]:
course_concept=course_concept.drop('concept_ccid',axis=1).drop_duplicates()

In [47]:
cb_course_concept = pd.merge(course_concept,concept,on="concept_id",how='inner')
cb_course_concept

,concept_id,course_id,concept_prerequisite,concept_name,concept_field
0,K_001定时器得电_控制科学与工程,C_681460,NaN,001 timer is powered,control science and engineering
1,K_002平面衍射强度_作物学,C_697288,NaN,002 plane diffraction intensity,crop science
2,K_0 part_计算机科学与技术,C_597208,NaN,part,computer science and technology
3,K_0 part_计算机科学与技术,C_694136,NaN,part,computer science and technology
4,K_0 part_计算机科学与技术,C_830568,NaN,part,computer science and technology
...,...,...,...,...,...
456029,K_座位信息_计算机科学与技术,C_682737,NaN,seat information,computer science and technology
456030,K_座椅_机械工程,C_696923,NaN,seats,mechanical engineering
456031,K_座椅_机械工程,C_854843,NaN,seats,mechanical engineering
456032,K_座椅_机械工程,C_680803,NaN,seats,mechanical engineering


In [48]:
cb_course_concept = cb_course_concept.groupby('course_id').agg(list).reset_index()
cb_course_concept

,course_id,concept_id,concept_prerequisite,concept_name,concept_field
0,C_1169394,"[K_abort_计算机科学与技术, K_area函数_计算机科学与技术, K_arrayo...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[abort, area function, arrayofpoints object, a...","[computer science and technology, computer sci..."
1,C_1410156,"[K_32位模式_计算机科学与技术, K_4g内存管理机制_计算机科学与技术, K_512字...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[32-bit mode, 4g memory management mechanism, ...","[computer science and technology, computer sci..."
2,C_1429045,"[K_24小时_临床医学, K_cochrane系统综述方法学小组_临床医学, K_inte...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[24 hours, cochrane systematic review methodol...","[clinical medicine, clinical medicine, clinica..."
3,C_1742805,"[K_16进制_计算机科学与技术, K_abort_计算机科学与技术, K_add函数_计算...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[hexadecimal, abort, add function, arctan func...","[computer science and technology, computer sci..."
4,C_1749733,"[K_2g 3g 4g_计算机科学与技术, K_+ 2_计算机科学与技术, K_3d spa...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2g 3g 4g, + 2, 3d space, groups, android, an ...","[computer science and technology, computer sci..."
...,...,...,...,...,...
882,C_890259,"[K_cad设计_管理科学与工程, K_core competence_管理科学与工程, K...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[cad design, core competence, internal environ...","[management science and engineering, managemen..."
883,C_901947,"[K_beats fast_公共卫生与预防医学, K_body contacts_公共卫生与...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[beats fast, body contacts, body or mind, chan...","[public health and preventive medicine, public..."
884,C_922854,"[K_安全出口_建筑学, K_安装门窗框门口_建筑学, K_案管内排水，_建筑学, K_案_...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[emergency exit, install door and window frame...","[architecture, architecture, architecture, arc..."
885,C_928746,"[K_32位宽_计算机科学与技术, K_5g赫兹_计算机科学与技术, K_802.11n_计...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[32-bit width, 5g hz, 802.11n, 80286, arm, asi...","[computer science and technology, computer sci..."


In [49]:
combine_data = pd.merge(combine_data,cb_course_concept,on='course_id',how='left')
combine_data

,user_id,course_id,enroll_time,course_name,course_prerequisites,course_about,course_field,school_id,school_name,school_about,...,total_watch_time,actual_watch_time,exercise_id,exercise_num_problem,doing_exercise_id,doing_num_correct,concept_id,concept_prerequisite,concept_name,concept_field
0,U_24,C_597314,1560360127,university chinese—analysis of northern song d...,,"at present, the talent gap in data science and...",[chinese language and literature],"[S_1, S_14]","[tsinghua university, southeast university]",[referred tsinghua short directly ministry edu...,...,NaN,NaN,"[Ex_10607, Ex_10647, Ex_10628, Ex_10711, Ex_10...","[1, 6, 4, 2, 4, 3, 2, 6, 3, 4, 5, 6, 4, 2, 2, ...",NaN,NaN,"[K_D_中国语言文学, K_爱国豪情_中国语言文学, K_爱国将领_中国语言文学, K_爱...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[d, patriotic pride, patriotic general, patrio...","[chinese language and literature, chinese lang..."
1,U_24,C_597208,1560784961,data mining: theory and algorithm,,the most interesting theory + the most useful ...,[computer science and technology],[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,NaN,NaN,"[Ex_2994, Ex_3099, Ex_3066, Ex_3021, Ex_3006, ...","[5, 3, 5, 3, 5, 6, 3, 3, 5, 5, 4, 5, 5, 2, 5, ...",NaN,NaN,"[K_0 part_计算机科学与技术, K_2 cities_计算机科学与技术, K_2d ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[part, cities, 2d space, matrix, + 2, 3d infor...","[computer science and technology, computer sci..."
2,U_24,C_682129,1570379031,software theory basics,,"this course introduces formal language, automa...",NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,NaN,NaN,"[Ex_265490, Ex_265468, Ex_265398, Ex_265475, E...","[4, 5, 5, 6, 6, 6, 7, 7, 6, 6, 6, 5, 7, 7, 6, 5]",NaN,NaN,"[K_ASCII_计算机科学与技术, K_cfl的判定性质_计算机科学与技术, K_CYK算...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[ascii, determination of the nature of cfl, cy...","[computer science and technology, computer sci..."
3,U_24,C_674910,1570557064,pattern aesthetics and creation,,[national premium course] this course provides...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,NaN,NaN,"[Ex_35603, Ex_35595, Ex_35599, Ex_35583, Ex_35...","[1, 1, 2, 3, 1, 1, 3, 2, 2, 2, 3]",NaN,NaN,NaN,NaN,NaN,NaN
4,U_24,C_707038,1570971985,organic chemistry,,the world's organic compounds are rich and col...,"[chemical engineering and technology, chemical]",[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,NaN,NaN,"[Ex_567007, Ex_566841, Ex_566866, Ex_567006, E...","[1, 6, 8, 1, 19, 8, 11, 7, 8, 9, 9, 1, 16, 5, ...",NaN,NaN,"[K_1,4-加成_化学, K_1-溴戊烷_化学, K_1-丁醇_化学, K_1-丁烯_化学...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1,4-addition, 1-bromopentane, 1-butanol, 1-bu...","[chemical, chemical, chemical, chemical, chemi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7473119,U_34712115,C_770738,1602487061,data structure (part 2),,breeze plan 2021 [first term recruitment] is i...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,NaN,NaN,"[Ex_1004448, Ex_1011216, Ex_1004394, Ex_784166...","[2, 1, 1, 1, 1, 2, 1, 3, 1, 16, 1, 3, 3, 2, 1,...",NaN,NaN,"[K_0号寄存器_计算机科学与技术, K_0号元素_计算机科学与技术, K_10进制_计算机...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[register 0, element 0, decimal, register 1, 8...","[computer science and technology, computer sci..."
7473120,U_34712115,C_676937,1603225930,advanced c++ language programming,,breeze plan 2021 [first term recruitment] is i...,[computer science and technology],[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,NaN,NaN,"[Ex_69254, Ex_69224, Ex_69293, Ex_69253, Ex_69...","[1, 2, 2, 3, 1, 2, 4, 5, 1, 1, 1, 1, 4, 1, 1, ...",NaN,NaN,"[K_abort_计算机科学与技术, K_area函数_计算机科学与技术, K_array对...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[abort, area function, array object, begin fun...","[computer science and tec

In [50]:
user_course_comment

,user_id,course_id,num_comments,num_replies,total_interactions,total_course_comments
0,U_24,C_597214,0,0,0,0
1,U_24,C_605512,0,0,0,1
2,U_24,C_597211,0,0,0,0
3,U_24,C_597314,0,0,0,0
4,U_24,C_597208,0,0,0,0
...,...,...,...,...,...,...
7591770,U_34712115,C_770738,0,0,0,339
7591771,U_34712115,C_676937,0,0,0,370
7591772,U_34712115,C_694136,0,0,0,1108
7591773,U_34712115,C_1738993,0,0,0,77


In [51]:
course_comment=user_course_comment[['course_id','total_course_comments']].drop_duplicates().reset_index()
course_comment=course_comment.rename(columns={'total_course_comments':'course_total_comments'})
course_comment=course_comment[['course_id','course_total_comments']]
course_comment

,course_id,course_total_comments
0,C_597214,0
1,C_605512,1
2,C_597211,0
3,C_597314,0
4,C_597208,0
...,...,...
3939,C_2275456,0
3940,C_2321310,0
3941,C_2334325,0
3942,C_2220132,0


In [52]:
combine_data = pd.merge(combine_data,course_comment,on='course_id',how='left')
combine_data

,user_id,course_id,enroll_time,course_name,course_prerequisites,course_about,course_field,school_id,school_name,school_about,...,actual_watch_time,exercise_id,exercise_num_problem,doing_exercise_id,doing_num_correct,concept_id,concept_prerequisite,concept_name,concept_field,course_total_comments
0,U_24,C_597314,1560360127,university chinese—analysis of northern song d...,,"at present, the talent gap in data science and...",[chinese language and literature],"[S_1, S_14]","[tsinghua university, southeast university]",[referred tsinghua short directly ministry edu...,...,NaN,"[Ex_10607, Ex_10647, Ex_10628, Ex_10711, Ex_10...","[1, 6, 4, 2, 4, 3, 2, 6, 3, 4, 5, 6, 4, 2, 2, ...",NaN,NaN,"[K_D_中国语言文学, K_爱国豪情_中国语言文学, K_爱国将领_中国语言文学, K_爱...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[d, patriotic pride, patriotic general, patrio...","[chinese language and literature, chinese lang...",0
1,U_24,C_597208,1560784961,data mining: theory and algorithm,,the most interesting theory + the most useful ...,[computer science and technology],[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,NaN,"[Ex_2994, Ex_3099, Ex_3066, Ex_3021, Ex_3006, ...","[5, 3, 5, 3, 5, 6, 3, 3, 5, 5, 4, 5, 5, 2, 5, ...",NaN,NaN,"[K_0 part_计算机科学与技术, K_2 cities_计算机科学与技术, K_2d ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[part, cities, 2d space, matrix, + 2, 3d infor...","[computer science and technology, computer sci...",0
2,U_24,C_682129,1570379031,software theory basics,,"this course introduces formal language, automa...",NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,NaN,"[Ex_265490, Ex_265468, Ex_265398, Ex_265475, E...","[4, 5, 5, 6, 6, 6, 7, 7, 6, 6, 6, 5, 7, 7, 6, 5]",NaN,NaN,"[K_ASCII_计算机科学与技术, K_cfl的判定性质_计算机科学与技术, K_CYK算...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[ascii, determination of the nature of cfl, cy...","[computer science and technology, computer sci...",13
3,U_24,C_674910,1570557064,pattern aesthetics and creation,,[national premium course] this course provides...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,NaN,"[Ex_35603, Ex_35595, Ex_35599, Ex_35583, Ex_35...","[1, 1, 2, 3, 1, 1, 3, 2, 2, 2, 3]",NaN,NaN,NaN,NaN,NaN,NaN,386
4,U_24,C_707038,1570971985,organic chemistry,,the world's organic compounds are rich and col...,"[chemical engineering and technology, chemical]",[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,NaN,"[Ex_567007, Ex_566841, Ex_566866, Ex_567006, E...","[1, 6, 8, 1, 19, 8, 11, 7, 8, 9, 9, 1, 16, 5, ...",NaN,NaN,"[K_1,4-加成_化学, K_1-溴戊烷_化学, K_1-丁醇_化学, K_1-丁烯_化学...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1,4-addition, 1-bromopentane, 1-butanol, 1-bu...","[chemical, chemical, chemical, chemical, chemi...",1876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7473119,U_34712115,C_770738,1602487061,data structure (part 2),,breeze plan 2021 [first term recruitment] is i...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,NaN,"[Ex_1004448, Ex_1011216, Ex_1004394, Ex_784166...","[2, 1, 1, 1, 1, 2, 1, 3, 1, 16, 1, 3, 3, 2, 1,...",NaN,NaN,"[K_0号寄存器_计算机科学与技术, K_0号元素_计算机科学与技术, K_10进制_计算机...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[register 0, element 0, decimal, register 1, 8...","[computer science and technology, computer sci...",339
7473120,U_34712115,C_676937,1603225930,advanced c++ language programming,,breeze plan 2021 [first term recruitment] is i...,[computer science and technology],[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,NaN,"[Ex_69254, Ex_69224, Ex_69293, Ex_69253, Ex_69...","[1, 2, 2, 3, 1, 2, 4, 5, 1, 1, 1, 1, 4, 1, 1, ...",NaN,NaN,"[K_abort_计算机科学与技术, K_area函数_计算机科学与技术, K_array对...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[abort, area function, array object, begin fun...","[computer science and techno

In [53]:
user_course_comment=user_course_comment[['user_id','course_id','num_comments','num_replies']]
user_course_comment=user_course_comment.rename(columns={
    'num_comments':'user_course_num_comment',
    'num_replies':'user_course_num_replies'})
user_course_comment

,user_id,course_id,user_course_num_comment,user_course_num_replies
0,U_24,C_597214,0,0
1,U_24,C_605512,0,0
2,U_24,C_597211,0,0
3,U_24,C_597314,0,0
4,U_24,C_597208,0,0
...,...,...,...,...
7591770,U_34712115,C_770738,0,0
7591771,U_34712115,C_676937,0,0
7591772,U_34712115,C_694136,0,0
7591773,U_34712115,C_1738993,0,0


In [54]:
combine_data = pd.merge(combine_data,user_course_comment,on=['user_id','course_id'],how='left')
combine_data

,user_id,course_id,enroll_time,course_name,course_prerequisites,course_about,course_field,school_id,school_name,school_about,...,exercise_num_problem,doing_exercise_id,doing_num_correct,concept_id,concept_prerequisite,concept_name,concept_field,course_total_comments,user_course_num_comment,user_course_num_replies
0,U_24,C_597314,1560360127,university chinese—analysis of northern song d...,,"at present, the talent gap in data science and...",[chinese language and literature],"[S_1, S_14]","[tsinghua university, southeast university]",[referred tsinghua short directly ministry edu...,...,"[1, 6, 4, 2, 4, 3, 2, 6, 3, 4, 5, 6, 4, 2, 2, ...",NaN,NaN,"[K_D_中国语言文学, K_爱国豪情_中国语言文学, K_爱国将领_中国语言文学, K_爱...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[d, patriotic pride, patriotic general, patrio...","[chinese language and literature, chinese lang...",0,0,0
1,U_24,C_597208,1560784961,data mining: theory and algorithm,,the most interesting theory + the most useful ...,[computer science and technology],[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[5, 3, 5, 3, 5, 6, 3, 3, 5, 5, 4, 5, 5, 2, 5, ...",NaN,NaN,"[K_0 part_计算机科学与技术, K_2 cities_计算机科学与技术, K_2d ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[part, cities, 2d space, matrix, + 2, 3d infor...","[computer science and technology, computer sci...",0,0,0
2,U_24,C_682129,1570379031,software theory basics,,"this course introduces formal language, automa...",NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[4, 5, 5, 6, 6, 6, 7, 7, 6, 6, 6, 5, 7, 7, 6, 5]",NaN,NaN,"[K_ASCII_计算机科学与技术, K_cfl的判定性质_计算机科学与技术, K_CYK算...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[ascii, determination of the nature of cfl, cy...","[computer science and technology, computer sci...",13,0,0
3,U_24,C_674910,1570557064,pattern aesthetics and creation,,[national premium course] this course provides...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[1, 1, 2, 3, 1, 1, 3, 2, 2, 2, 3]",NaN,NaN,NaN,NaN,NaN,NaN,386,0,0
4,U_24,C_707038,1570971985,organic chemistry,,the world's organic compounds are rich and col...,"[chemical engineering and technology, chemical]",[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[1, 6, 8, 1, 19, 8, 11, 7, 8, 9, 9, 1, 16, 5, ...",NaN,NaN,"[K_1,4-加成_化学, K_1-溴戊烷_化学, K_1-丁醇_化学, K_1-丁烯_化学...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1,4-addition, 1-bromopentane, 1-butanol, 1-bu...","[chemical, chemical, chemical, chemical, chemi...",1876,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7473119,U_34712115,C_770738,1602487061,data structure (part 2),,breeze plan 2021 [first term recruitment] is i...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[2, 1, 1, 1, 1, 2, 1, 3, 1, 16, 1, 3, 3, 2, 1,...",NaN,NaN,"[K_0号寄存器_计算机科学与技术, K_0号元素_计算机科学与技术, K_10进制_计算机...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[register 0, element 0, decimal, register 1, 8...","[computer science and technology, computer sci...",339,0,0
7473120,U_34712115,C_676937,1603225930,advanced c++ language programming,,breeze plan 2021 [first term recruitment] is i...,[computer science and technology],[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[1, 2, 2, 3, 1, 2, 4, 5, 1, 1, 1, 1, 4, 1, 1, ...",NaN,NaN,"[K_abort_计算机科学与技术, K_area函数_计算机科学与技术, K_array对...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[abort, area function, array object, begin fun...","[computer science and technology, computer sci...",370,0,0
7473121,U_34712115,C_694136,1603226018,data mining: theory and algorithm,,the most interesting theory + the most useful ...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[4, 1, 7, 4, 6, 5, 4, 5, 4, 7, 5, 5, 3, 6, 5, ...",NaN,NaN,"[K_0 part_计算机科学与技术, K_2 cities_计算机科学与技术, K_2d ..

In [55]:
print('Saving...')

Saving...


In [56]:
combine_data = combine_data[combine_data['video_id'].apply(lambda x: isinstance(x, list) and len(x) > 0) | 
                 combine_data['exercise_id'].apply(lambda x: isinstance(x, list) and len(x) > 0)]

In [57]:
def filter_user_course(df_user_course, min_courses_per_user=5, min_users_per_course=5):
    prev_shape = (0, 0)
    current_shape = df_user_course.shape

    while current_shape != prev_shape:
        prev_shape = current_shape
        print(f'shape: {prev_shape}')
        # Lọc user có số khóa học >= min_courses_per_user
        user_counts = df_user_course['user_id'].value_counts()
        valid_users = user_counts[user_counts >= min_courses_per_user].index
        df_user_course = df_user_course[df_user_course['user_id'].isin(valid_users)]

        # Lọc course có số user >= min_users_per_course
        course_counts = df_user_course['course_id'].value_counts()
        valid_courses = course_counts[course_counts >= min_users_per_course].index
        df_user_course = df_user_course[df_user_course['course_id'].isin(valid_courses)]
        
        current_shape = df_user_course.shape
    print(f'current: {current_shape}')
    return df_user_course
combine_data = filter_user_course(combine_data)

shape: (7337772, 34)
shape: (7284552, 34)
shape: (7284500, 34)
current: (7284500, 34)


In [58]:
combine_data.head()

,user_id,course_id,enroll_time,course_name,course_prerequisites,course_about,course_field,school_id,school_name,school_about,...,exercise_num_problem,doing_exercise_id,doing_num_correct,concept_id,concept_prerequisite,concept_name,concept_field,course_total_comments,user_course_num_comment,user_course_num_replies
0,U_24,C_597314,1560360127,university chinese—analysis of northern song d...,,"at present, the talent gap in data science and...",[chinese language and literature],"[S_1, S_14]","[tsinghua university, southeast university]",[referred tsinghua short directly ministry edu...,...,"[1, 6, 4, 2, 4, 3, 2, 6, 3, 4, 5, 6, 4, 2, 2, ...",NaN,NaN,"[K_D_中国语言文学, K_爱国豪情_中国语言文学, K_爱国将领_中国语言文学, K_爱...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[d, patriotic pride, patriotic general, patrio...","[chinese language and literature, chinese lang...",0,0,0
1,U_24,C_597208,1560784961,data mining: theory and algorithm,,the most interesting theory + the most useful ...,[computer science and technology],[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[5, 3, 5, 3, 5, 6, 3, 3, 5, 5, 4, 5, 5, 2, 5, ...",NaN,NaN,"[K_0 part_计算机科学与技术, K_2 cities_计算机科学与技术, K_2d ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[part, cities, 2d space, matrix, + 2, 3d infor...","[computer science and technology, computer sci...",0,0,0
2,U_24,C_682129,1570379031,software theory basics,,"this course introduces formal language, automa...",NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[4, 5, 5, 6, 6, 6, 7, 7, 6, 6, 6, 5, 7, 7, 6, 5]",NaN,NaN,"[K_ASCII_计算机科学与技术, K_cfl的判定性质_计算机科学与技术, K_CYK算...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[ascii, determination of the nature of cfl, cy...","[computer science and technology, computer sci...",13,0,0
3,U_24,C_674910,1570557064,pattern aesthetics and creation,,[national premium course] this course provides...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[1, 1, 2, 3, 1, 1, 3, 2, 2, 2, 3]",NaN,NaN,NaN,NaN,NaN,NaN,386,0,0
4,U_24,C_707038,1570971985,organic chemistry,,the world's organic compounds are rich and col...,"[chemical engineering and technology, chemical]",[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[1, 6, 8, 1, 19, 8, 11, 7, 8, 9, 9, 1, 16, 5, ...",NaN,NaN,"[K_1,4-加成_化学, K_1-溴戊烷_化学, K_1-丁醇_化学, K_1-丁烯_化学...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1,4-addition, 1-bromopentane, 1-butanol, 1-bu...","[chemical, chemical, chemical, chemical, chemi...",1876,0,0


In [59]:
combine_data.to_hdf('combine_data.h5', key='df', mode='w')

<ipython-input-59-25ae01b55dd7>:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['user_id', 'course_id', 'course_name', 'course_prerequisites',
       'course_about', 'course_field', 'school_id', 'school_name',
       'school_about', 'school_motto', 'teacher_id', 'teacher_name',
       'teacher_about', 'teacher_job_title', 'teacher_org_name', 'video_id',
       'video_ccid', 'video_duration', 'watch_id', 'total_watch_time',
       'actual_watch_time', 'exercise_id', 'exercise_num_problem',
       'doing_exercise_id', 'doing_num_correct', 'concept_id',
       'concept_prerequisite', 'concept_name', 'concept_field'],
      dtype='object')]

  combine_data.to_hdf('combine_data.h5', key='df', mode='w')


In [60]:
print('Completed!')

Completed!


In [61]:
del combine_data

In [62]:
combine_data = pd.read_hdf('combine_data.h5', key='df')
combine_data

,user_id,course_id,enroll_time,course_name,course_prerequisites,course_about,course_field,school_id,school_name,school_about,...,exercise_num_problem,doing_exercise_id,doing_num_correct,concept_id,concept_prerequisite,concept_name,concept_field,course_total_comments,user_course_num_comment,user_course_num_replies
0,U_24,C_597314,1560360127,university chinese—analysis of northern song d...,,"at present, the talent gap in data science and...",[chinese language and literature],"[S_1, S_14]","[tsinghua university, southeast university]",[referred tsinghua short directly ministry edu...,...,"[1, 6, 4, 2, 4, 3, 2, 6, 3, 4, 5, 6, 4, 2, 2, ...",NaN,NaN,"[K_D_中国语言文学, K_爱国豪情_中国语言文学, K_爱国将领_中国语言文学, K_爱...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[d, patriotic pride, patriotic general, patrio...","[chinese language and literature, chinese lang...",0,0,0
1,U_24,C_597208,1560784961,data mining: theory and algorithm,,the most interesting theory + the most useful ...,[computer science and technology],[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[5, 3, 5, 3, 5, 6, 3, 3, 5, 5, 4, 5, 5, 2, 5, ...",NaN,NaN,"[K_0 part_计算机科学与技术, K_2 cities_计算机科学与技术, K_2d ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[part, cities, 2d space, matrix, + 2, 3d infor...","[computer science and technology, computer sci...",0,0,0
2,U_24,C_682129,1570379031,software theory basics,,"this course introduces formal language, automa...",NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[4, 5, 5, 6, 6, 6, 7, 7, 6, 6, 6, 5, 7, 7, 6, 5]",NaN,NaN,"[K_ASCII_计算机科学与技术, K_cfl的判定性质_计算机科学与技术, K_CYK算...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[ascii, determination of the nature of cfl, cy...","[computer science and technology, computer sci...",13,0,0
3,U_24,C_674910,1570557064,pattern aesthetics and creation,,[national premium course] this course provides...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[1, 1, 2, 3, 1, 1, 3, 2, 2, 2, 3]",NaN,NaN,NaN,NaN,NaN,NaN,386,0,0
4,U_24,C_707038,1570971985,organic chemistry,,the world's organic compounds are rich and col...,"[chemical engineering and technology, chemical]",[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[1, 6, 8, 1, 19, 8, 11, 7, 8, 9, 9, 1, 16, 5, ...",NaN,NaN,"[K_1,4-加成_化学, K_1-溴戊烷_化学, K_1-丁醇_化学, K_1-丁烯_化学...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1,4-addition, 1-bromopentane, 1-butanol, 1-bu...","[chemical, chemical, chemical, chemical, chemi...",1876,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7473119,U_34712115,C_770738,1602487061,data structure (part 2),,breeze plan 2021 [first term recruitment] is i...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[2, 1, 1, 1, 1, 2, 1, 3, 1, 16, 1, 3, 3, 2, 1,...",NaN,NaN,"[K_0号寄存器_计算机科学与技术, K_0号元素_计算机科学与技术, K_10进制_计算机...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[register 0, element 0, decimal, register 1, 8...","[computer science and technology, computer sci...",339,0,0
7473120,U_34712115,C_676937,1603225930,advanced c++ language programming,,breeze plan 2021 [first term recruitment] is i...,[computer science and technology],[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[1, 2, 2, 3, 1, 2, 4, 5, 1, 1, 1, 1, 4, 1, 1, ...",NaN,NaN,"[K_abort_计算机科学与技术, K_area函数_计算机科学与技术, K_array对...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[abort, area function, array object, begin fun...","[computer science and technology, computer sci...",370,0,0
7473121,U_34712115,C_694136,1603226018,data mining: theory and algorithm,,the most interesting theory + the most useful ...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[4, 1, 7, 4, 6, 5, 4, 5, 4, 7, 5, 5, 3, 6, 5, ...",NaN,NaN,"[K_0 part_计算机科学与技术, K_2 cities_计算机科学与技术, K_2d ..

In [63]:
combine_data.isnull().mean()

user_id                    0.000000
course_id                  0.000000
enroll_time                0.000000
course_name                0.000000
course_prerequisites       0.000000
course_about               0.000000
course_field               0.731191
school_id                  0.003643
school_name                0.003643
school_about               0.003643
school_motto               0.003643
teacher_id                 0.001367
teacher_name               0.001367
teacher_about              0.001367
teacher_job_title          0.001367
teacher_org_name           0.001367
video_id                   0.379174
video_ccid                 0.379174
video_duration             0.379174
user_gender                0.000000
watch_id                   0.999437
total_watch_time           0.999437
actual_watch_time          0.999437
exercise_id                0.019379
exercise_num_problem       0.019379
doing_exercise_id          0.996887
doing_num_correct          0.996887
concept_id                 0